In [152]:


import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import numpy as np


import data_generator


In [153]:
# generate the data

x_set = [];
y_set = [];

for i in range(0,20):
    graphset = data_generator.get_graphset(i);
    x_set.append(graphset[0]);
    y_set.append(graphset[1][0]);




In [154]:
# split the data into training and testing sets


x_set = np.array(x_set);
y_set = np.array(y_set);

half = int(len(x_set)/2);

print(f'half: {half}');
x_train = x_set[0:half];
x_test = x_set[half:];

y_train = y_set[0:half];
y_test = y_set[half:];



half: 10


In [155]:
# print some data diagnostics to ensure we correctly obtained data sets

print(f"x_set shape: {x_set.shape}");
print(f"y_set shape: {y_set.shape}");

print("number of x data points:");
print(len(x_train[0:]));


print("number of y data points:");
print(len(y_train[0:]));

assert(len(x_train) == len(y_train));
assert(len(x_test) == len(y_test));
assert(x_set.shape == y_set.shape);

print(f"\nNumber of training graph-solution sets: {len(x_train)}")


print(f"Number of testing graph-solution sets: {len(x_test)}")



print(f"\nDataset shape: {x_set.shape}")



x_set shape: (20, 5000)
y_set shape: (20, 5000)
number of x data points:
10
number of y data points:
10

Number of training graph-solution sets: 10
Number of testing graph-solution sets: 10

Dataset shape: (20, 5000)


In [169]:
#build the model by stacking layers; choose optimizer and loss function. Here we set the activation function to a 'relu'


model = tf.keras.models.Sequential()



#return_sequences: whether the next layer is another RNN
#model.add(LSTM(128,input_shape=( x_train.shape[1:] ), activation='relu',return_sequences=True)); 

model.add(Dense(5000,activation='relu'));
model.add(Dropout(0.2));
model.add(Dense(5000,activation='relu'));
model.add(Dropout(0.2));
model.add(Dense(5000,activation='relu'));
model.add(Dropout(0.2));
model.add(Dense(5000,activation='relu'));
model.add(Dropout(0.2));
model.add(Dense(5000,activation='softmax'));


print("model built");

model built


In [170]:
# get the predictions for the second datapoint
predictions = model(x_train[1:])

# print the raw predictions for that datapoint. Note that these are "logits" or "log-odds" scores, which is related to the probability, 
# and also is the inverse of the sigmoid function
print("logits prediction: ");
print(predictions);

# we can use the softmax function to convert these logits to probabilities for each preciction class
print("odds prediction: ");
print(tf.nn.softmax(predictions).numpy());



logits prediction: 
tf.Tensor(
[[0.00025317 0.00017885 0.00014997 ... 0.00013215 0.00014596 0.0002594 ]
 [0.00026008 0.00017721 0.00014699 ... 0.00012164 0.00013868 0.00026708]
 [0.00026001 0.00017725 0.00014701 ... 0.00012178 0.00013875 0.000267  ]
 ...
 [0.00026001 0.00017725 0.00014701 ... 0.00012178 0.00013875 0.000267  ]
 [0.00026001 0.00017725 0.00014701 ... 0.00012178 0.00013875 0.000267  ]
 [0.00026001 0.00017725 0.00014701 ... 0.00012178 0.00013875 0.000267  ]], shape=(9, 5000), dtype=float32)
odds prediction: 
[[0.00020001 0.0002     0.00019999 ... 0.00019999 0.00019999 0.00020001]
 [0.00020001 0.0002     0.00019999 ... 0.00019998 0.00019999 0.00020001]
 [0.00020001 0.0002     0.00019999 ... 0.00019998 0.00019999 0.00020001]
 ...
 [0.00020001 0.0002     0.00019999 ... 0.00019998 0.00019999 0.00020001]
 [0.00020001 0.0002     0.00019999 ... 0.00019998 0.00019999 0.00020001]
 [0.00020001 0.0002     0.00019999 ... 0.00019998 0.00019999 0.00020001]]


In [174]:
# get the loss function as a SparseCategoricalCrossentropy loss from tf.keras.losses. 
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False);

print("loss:");
print(loss_fn(y_train[1:], predictions).numpy());

loss:


ValueError: Shape mismatch: The shape of labels (received (45000,)) should equal the shape of logits except for the last dimension (received (9, 5000)).

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-3,decay=1e-5);

model.compile(loss=loss_fn,optimizer=opt,metrics=['accuracy']);


In [ ]:
model.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test));

Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: -1691929620250624.0000 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 1s 748ms/step - loss: -32759983309324288.0000 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 1s 759ms/step - loss: -237330581287862272.0000 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 1s 753ms/step - loss: -957727060753645568.0000 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 1s 788ms/step - loss: -2365119055456632832.0000 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 1s 795ms/step - loss: -6067680206649294848.0000 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accur